# Cargar y explorar el conjunto de datos
Cargue el conjunto de datos de consumo de energía eléctrica del hogar individual y realice un análisis exploratorio de datos (EDA) para comprender los datos.

In [ ]:
# Cargar y explorar el conjunto de datos

import pandas as pd
import matplotlib.pyplot as plt

# Cargar el conjunto de datos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
data = pd.read_csv(url, sep=';', parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'])

# Mostrar las primeras filas del conjunto de datos
print(data.head())

# Información del conjunto de datos
print(data.info())

# Estadísticas descriptivas
print(data.describe())

# Visualización de datos faltantes
plt.figure(figsize=(10, 6))
plt.imshow(data.isna(), aspect='auto', cmap='gray')
plt.xlabel('Columnas')
plt.ylabel('Filas')
plt.title('Visualización de datos faltantes')
plt.show()

# Visualización de la serie temporal de consumo de energía global activa
plt.figure(figsize=(15, 6))
plt.plot(data['datetime'], data['Global_active_power'], label='Consumo de energía global activa')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Consumo de energía global activa a lo largo del tiempo')
plt.legend()
plt.show()

# Preprocesamiento de datos
Preprocese los datos limpiando los valores faltantes, normalizando los datos y preparando los datos para su uso en modelos de series temporales.

In [ ]:
# Preprocesamiento de datos

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Eliminar filas con valores faltantes
data.dropna(inplace=True)

# Convertir la columna 'Global_active_power' a tipo float
data['Global_active_power'] = data['Global_active_power'].astype(float)

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data['Global_active_power'] = scaler.fit_transform(data['Global_active_power'].values.reshape(-1, 1))

# Preparar los datos para modelos de series temporales
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

# Definir el paso de tiempo
time_step = 100

# Crear el conjunto de datos
X, y = create_dataset(data['Global_active_power'].values, time_step)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# Redimensionar los datos para que sean compatibles con LSTM y GRU
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Implementación del modelo LSTM
Implemente un modelo de red LSTM para predecir el consumo de energía eléctrica.

In [ ]:
# Implementación del modelo LSTM

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Crear el modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dense(1))

# Compilar el modelo
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history_lstm = model_lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, verbose=1)

# Realizar predicciones
train_predict_lstm = model_lstm.predict(X_train)
test_predict_lstm = model_lstm.predict(X_test)

# Invertir la normalización para obtener los valores originales
train_predict_lstm = scaler.inverse_transform(train_predict_lstm)
test_predict_lstm = scaler.inverse_transform(test_predict_lstm)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Visualizar las predicciones
plt.figure(figsize=(15, 6))
plt.plot(data['datetime'][:len(y_train)], y_train, label='Datos de entrenamiento reales')
plt.plot(data['datetime'][:len(train_predict_lstm)], train_predict_lstm, label='Predicciones de entrenamiento LSTM')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(y_test)], y_test, label='Datos de prueba reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_lstm)], test_predict_lstm, label='Predicciones de prueba LSTM')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Predicciones del modelo LSTM')
plt.legend()
plt.show()

# Implementación del modelo GRU
Implemente un modelo de red GRU para predecir el consumo de energía eléctrica.

In [ ]:
# Implementación del modelo GRU

from tensorflow.keras.layers import GRU

# Crear el modelo GRU
model_gru = Sequential()
model_gru.add(GRU(50, return_sequences=True, input_shape=(time_step, 1)))
model_gru.add(GRU(50, return_sequences=False))
model_gru.add(Dense(1))

# Compilar el modelo
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history_gru = model_gru.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, verbose=1)

# Realizar predicciones
train_predict_gru = model_gru.predict(X_train)
test_predict_gru = model_gru.predict(X_test)

# Invertir la normalización para obtener los valores originales
train_predict_gru = scaler.inverse_transform(train_predict_gru)
test_predict_gru = scaler.inverse_transform(test_predict_gru)

# Visualizar las predicciones
plt.figure(figsize=(15, 6))
plt.plot(data['datetime'][:len(y_train)], y_train, label='Datos de entrenamiento reales')
plt.plot(data['datetime'][:len(train_predict_gru)], train_predict_gru, label='Predicciones de entrenamiento GRU')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(y_test)], y_test, label='Datos de prueba reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_gru)], test_predict_gru, label='Predicciones de prueba GRU')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Predicciones del modelo GRU')
plt.legend()
plt.show()

# Evaluación del modelo
Evalúe el rendimiento de ambos modelos utilizando métricas como el error cuadrático medio (RMSE) y el error absoluto medio (MAE).

In [ ]:
# Evaluación del modelo

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calcular RMSE y MAE para el modelo LSTM
rmse_lstm = np.sqrt(mean_squared_error(y_test, test_predict_lstm))
mae_lstm = mean_absolute_error(y_test, test_predict_lstm)

# Calcular RMSE y MAE para el modelo GRU
rmse_gru = np.sqrt(mean_squared_error(y_test, test_predict_gru))
mae_gru = mean_absolute_error(y_test, test_predict_gru)

# Mostrar los resultados
print(f"Modelo LSTM - RMSE: {rmse_lstm}, MAE: {mae_lstm}")
print(f"Modelo GRU - RMSE: {rmse_gru}, MAE: {mae_gru}")

# Comparar los resultados en una tabla
import pandas as pd

results = pd.DataFrame({
    'Modelo': ['LSTM', 'GRU'],
    'RMSE': [rmse_lstm, rmse_gru],
    'MAE': [mae_lstm, mae_gru]
})

print(results)

# Visualizar las predicciones de ambos modelos
plt.figure(figsize=(15, 6))
plt.plot(data['datetime'][:len(y_train)], y_train, label='Datos de entrenamiento reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(y_test)], y_test, label='Datos de prueba reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_lstm)], test_predict_lstm, label='Predicciones de prueba LSTM')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_gru)], test_predict_gru, label='Predicciones de prueba GRU')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Comparación de predicciones de modelos LSTM y GRU')
plt.legend()
plt.show()

# Comparación de modelos
Compare los resultados obtenidos por los modelos LSTM y GRU, analizando sus fortalezas y debilidades en este contexto específico.

In [ ]:
# Comparación de modelos

# Calcular RMSE y MAE para el modelo LSTM
rmse_lstm = np.sqrt(mean_squared_error(y_test, test_predict_lstm))
mae_lstm = mean_absolute_error(y_test, test_predict_lstm)

# Calcular RMSE y MAE para el modelo GRU
rmse_gru = np.sqrt(mean_squared_error(y_test, test_predict_gru))
mae_gru = mean_absolute_error(y_test, test_predict_gru)

# Mostrar los resultados
print(f"Modelo LSTM - RMSE: {rmse_lstm}, MAE: {mae_lstm}")
print(f"Modelo GRU - RMSE: {rmse_gru}, MAE: {mae_gru}")

# Comparar los resultados en una tabla
results = pd.DataFrame({
    'Modelo': ['LSTM', 'GRU'],
    'RMSE': [rmse_lstm, rmse_gru],
    'MAE': [mae_lstm, mae_gru]
})

print(results)

# Visualizar las predicciones de ambos modelos
plt.figure(figsize=(15, 6))
plt.plot(data['datetime'][:len(y_train)], y_train, label='Datos de entrenamiento reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(y_test)], y_test, label='Datos de prueba reales')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_lstm)], test_predict_lstm, label='Predicciones de prueba LSTM')
plt.plot(data['datetime'][len(y_train):len(y_train) + len(test_predict_gru)], test_predict_gru, label='Predicciones de prueba GRU')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Comparación de predicciones de modelos LSTM y GRU')
plt.legend()
plt.show()